In [ ]:
def get_images(images_path):
    # speichert Bilder als numpy array

    array_images = []
    train_or_test_folder = os.listdir(images_path)

    for folder in natsorted(train_or_test_folder):
        single_folder = os.path.join(images_path, folder)

        for file in os.listdir(single_folder):
            filepath = os.path.join(single_folder, file)

            if filepath.lower().endswith(('.jpeg', '.jpg')):
                image = cv2.resize(cv2.imread(filepath), (64, 64))  # resize Größe bestimmt durch vortainiertes Netz
                array_images.append(image)

    images = np.array(array_images)
    
    return images

In [ ]:
def get_label(images_path):
    # speichert Lables als numpy array

    array_label = []
    for folder in natsorted(os.listdir(images_path)):
        label = int(folder)

        for file in os.listdir(os.path.join(images_path, folder)):
            array_label.append(label)
    labels = np.array(array_label)
        
    return labels

In [ ]:
images = get_images("../../02_data_crop/train") # train images
labels = get_label("../../02_data_crop/train")  # train labels

X_eval = get_images("../../02_data_crop/test")  # Evaluierungs Bilder
y_eval = get_label("../../02_data_crop/test")   # Evaluierungs Labels

## multilineare Regression

In [ ]:
import tensorflow.contrib.learn as learn
import tf


dset = learn.datasets.mnist.read_data_sets('MNIST-data', one_hot=True)
print('Trainingsbilder: ',dset.train.images.shape)            # (55000,784)
print('Validierungsbilder: ',dset.validation.images.shape)    # (5000, 784)
print('Testbilder: ',dset.test.images.shape)                  # (10000, 784)

image_holder = tf.placeholder(tf.float32, [None, 784])
label_holder = tf.placeholder(tf.float32, [None, 10])

m = tf.Variable(tf.zeros([784, 27]))
b = tf.Variable(tf.zeros([27]))

loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=tf.matmul(image_holder, m) + b, labels= label_holder))

learning_rate = 0.01
num_epochs = 25
batch_size = 100
num_batches = int(dset.train.num_examples/batch_size)
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)

    for epoch in range (num_epochs):
        for batch in range(num_batches):
            image_batch, label_batch = dset.train.next_batch(batch_size)
            _, lossVal = sess.run([optimizer, loss],
                                  feed_dict={image_holder: image_batch,
                                             label_batch: label_batch})
            
    print('Finaler Verlust: ', lossVal)

## Tuning

Eingaben standisieren

In [ ]:
input_data = tf.constant([1., 3., 5., 7., 9.])
stat_mean, stat_var = tf.nn.moments(input_data, 0)
standard_data = tf.nn.batch_normalization(input_data, stat_mean, stat_var, 0., 1., 0.0001, name=None)

Gewichte initialisieren